# 3. Diversity

> **Goal:** Compare the samples according to their diversity.

---

**Overview**

This section examines the microbial diversity of the given samples, focusing on both within-sample complexity (alpha diversity) and between-sample differences (beta diversity). These measures allow the assessment of how microbial community structure varies with different metadata attributes.

The workflow is organized into three key steps:

1. **Sampling depth assessment**  
   Sequencing depths are inspected and an appropriate threshold is defined.

2. **Bootstrapping on Euler**  
   To achieve a sufficient number of iterations, the bootstrapping of the k-mer–based diversity calculation is performed on Euler.

3. **Diversity computation and statistical testing**  
   - **Alpha diversity:** Used to evaluate richness or evenness within individual samples.  
   - **Beta diversity:** Distance metrics such as Bray–Curtis and Jaccard quantify community differences between samples.  
     The analysis first applies **QIIME’s `beta-group-significance`**, which tests for group-level differences in beta diversity.  
   - **Follow-up tests:**  
     - **PERMANOVA (adonis):** Applied to determine whether any detected significance persists when accounting for multiple covariates (multivariable effects).  
     - **PERMDISP** (`beta-group-significance` with the `--permdisp` option): Used to assess whether observed differences are driven by heterogeneous dispersion rather than true shifts in community composition.

Together, these steps provide a robust framework for identifying which metadata attributes are associated with meaningful differences in microbial diversity.

---

**Table of Contents**

- [3.1 Import packages](#3.1-Import-packages)
- [3.2 Import the data](#3.2-Import-the-data)
- [3.3 Determination of the sampling depth](#3.3-Determination-of-the-sampling-depth)
- [3.4 Euler](#3.4-Euler)
    - [3.4.1 Import files](#3.4.1-Import-files)
    - [3.4.2 Bootstrapping script](#3.4.2-Bootstrapping-script)
- [3.5 Diversity](#3.5-Diversity)
    - [3.5.1 Alpha diversity](#3.5.1-Alpha-diversity)
    - [3.5.2 Beta diversity](#3.5.2-Beta-diversity)
        - [3.5.2.1 Bray-Curtis](#3.5.2.1-Bray-Curtis)
        - [3.5.2.1 Jaccard](#3.5.2.1-Jaccard)

## 3.1 Import packages

In [1]:
# Importing all required packages at the start of the notebook
import IPython

from qiime2 import Visualization

import pandas as pd

## 3.2 Import the data

In [2]:
# Location
data_dir = "Project_data/Diversity"
! mkdir -p "$data_dir"

## 3.3 Determination of the sampling depth

In [3]:
# Create a summary table
! qiime feature-table summarize \
    --i-table Project_data/Taxonomy/table_filtered.qza \
    --m-sample-metadata-file Project_data/Metadata/updated_fungut_metadata.tsv \
    --o-visualization $data_dir/table_filtered.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/table_filtered.qzv


In [4]:
Visualization.load(f"{data_dir}/table_filtered.qzv")

<visualization: Visualization uuid: 80dee0b4-74ce-4b44-83b3-446cc04c9420>

In [5]:
# Determine the sampling depth
! qiime diversity alpha-rarefaction \
    --i-table Project_data/Taxonomy/table_filtered.qza \
    --p-max-depth 80000 \
    --m-metadata-file Project_data/Metadata/updated_fungut_metadata.tsv \
    --o-visualization $data_dir/alpha-rarefaction.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/alpha-rarefaction.qzv


In [6]:
Visualization.load(f"{data_dir}/alpha-rarefaction.qzv")

<visualization: Visualization uuid: 8623b024-3f74-4560-9b3e-2c0142b0cfe3>

According to alpha rarefication, a sampling depth of 20.000 was chosen, since the Shannon and observed feature metrics start to plateau at this point. Referring to the feature table for this sampling depth results in a retention of 2.720.000 reads (40.81%) across 136 samples (90.67%).

## 3.4 Euler
The diversity analysis was performed using the `q2-boots` plugin for QIIME2. According to the [documentation](https://f1000research.com/articles/14-87/v1) of the plugin any iterations higher than 100 do not scale well. Therefore, the analysis was run on the Euler cluster with `n = 1000` to ensure robust results. As this plugin was not included in the previously installed MOSHPIT distribution, the Amplicon distribution had to be installed additionally via Miniconda.

### 3.4.1 Import files
As with the 2.Taxonomy script, the files required to run the bootstrapping on Euler were uploaded to Polybox for download by the script running on Euler.

### 3.4.2 Bootstraping script
The following script was run on Euler.

```bash
#!/bin/bash
#SBATCH --job-name=bootstraping
#SBATCH --time=24:00:00
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=4
#SBATCH --mem-per-cpu=32G
#SBATCH --output=bootstraping_%j.out
#SBATCH --error=bootstraping_%j.err
#SBATCH --mail-type=END,FAIL

# Activate conda
source ~/miniconda3/etc/profile.d/conda.sh
conda activate qiime2-amplicon-2025.10

# Data folder
data_dir="ProjectData"


# Download the meta data and reads
module load eth_proxy

# table_filtered
wget --content-disposition -nc --progress=dot:giga -P "$data_dir" https://polybox.ethz.ch/index.php/s/dpP6wHGcgWe2ncd/download
# rep-seqs_filtered
wget --content-disposition -nc --progress=dot:giga -P "$data_dir" https://polybox.ethz.ch/index.php/s/Kkb47x6PgmesMgt/download
# updated_fungut_metadata
wget --content-disposition -nc --progress=dot:giga -P "$data_dir" https://polybox.ethz.ch/index.php/s/WdQ3GPTSEC6qqZm/download

echo "Download done!"

# Run the bootstraping
qiime boots kmer-diversity \
  --i-table $data_dir/table_filtered.qza \
  --i-sequences $data_dir/rep-seqs_filtered.qza \
  --m-metadata-file $data_dir/updated_fungut_metadata.tsv\
  --p-sampling-depth 20000 \
  --p-n 1000 \
  --p-replacement \
  --p-alpha-average-method median \
  --p-beta-average-method medoid \
  --output-dir $data_dir/boots-kmer-diversity

echo "Bootstraping done!"
```

## 3.5 Diversity
The files created by the script on Euler were downloaded and uploaded to Polybox in order to be accessible for this script.

In [7]:
%%bash -s $data_dir

# Shannon
wget --content-disposition -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/XewRqjqBxPjsBMK/download
# Scatter plot
wget --content-disposition -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/SbRQpewZm9LDcrW/download
# Bray-Curtis distance matrix
wget --content-disposition -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/X689NN8ziCxCtaM/download
# Jaccard distance matrix
wget --content-disposition -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/jJ8MQj3oBx9ckyw/download

chmod -R +rxw "$1"

--2025-12-12 14:30:33--  https://polybox.ethz.ch/index.php/s/XewRqjqBxPjsBMK/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
--2025-12-12 14:30:33--  https://polybox.ethz.ch/index.php/s/SbRQpewZm9LDcrW/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
--2025-12-12 14:30:34--  https://polybox.ethz.ch/index.php/s/X689NN8ziCxCtaM/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
--2025-12-12 14:30:34--  https://polybox.ethz.ch/index.php/s/jJ8MQj3oBx9ckyw/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox

### 3.5.1 Alpha diversity

In [8]:
! qiime diversity alpha-group-significance \
    --i-alpha-diversity $data_dir/shannon.qza \
    --m-metadata-file $data_dir/../Metadata/updated_fungut_metadata.tsv \
    --o-visualization $data_dir/alpha_group_significance.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/alpha_group_significance.qzv


In [9]:
Visualization.load(f"{data_dir}/alpha_group_significance.qzv")

<visualization: Visualization uuid: be3d9503-20b4-4227-bdf2-356fe433ad8b>

### 3.5.2 Beta diversity
The Beta diversity was run with both distance matricies (Bray-Curtis & Jaccard) obtained during the boot strapping

In [10]:
Visualization.load(f"{data_dir}/scatter_plot.qzv")

<visualization: Visualization uuid: 7bde3219-d0e3-4425-b4e5-e489877a4d65>

#### 3.5.2.1 Bray-Curtis

##### Beta-group-significance

In [11]:
meta_cols = ["age_range", "sex_sample", "diet_type_sample", "ibd_sample", "gluten_sample", "continent", "bmi_category", "urban_category"]

for col in meta_cols:
    output_name = f"{data_dir}/bray_curtis-{col}-significance.qzv"
    print(f"Running for column: {col}")

    ! qiime diversity beta-group-significance \
        --i-distance-matrix $data_dir/braycurtis.qza \
        --m-metadata-file $data_dir/../Metadata/updated_fungut_metadata.tsv \
        --m-metadata-column {col} \
        --p-permutations 9999 \
        --p-pairwise \
        --o-visualization {output_name}
    
# Errors with country_sample (some unique values), bmi_sample (numeric type) 

Running for column: age_range
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/bray_curtis-age_range-significance.qzv
Running for column: sex_sample
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/bray_curtis-sex_sample-significance.qzv
Running for column: diet_type_sample
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarnin

In [12]:
Visualization.load(f"{data_dir}/bray_curtis-age_range-significance.qzv")

<visualization: Visualization uuid: d57b94cd-cea4-4824-8347-7dd2b2207963>

In [13]:
Visualization.load(f"{data_dir}/bray_curtis-sex_sample-significance.qzv")

<visualization: Visualization uuid: 720d0983-5598-411a-8f62-af5b3a2a37cf>

In [14]:
Visualization.load(f"{data_dir}/bray_curtis-diet_type_sample-significance.qzv")

<visualization: Visualization uuid: 41c760ac-e18b-4c38-970b-ad985334ddaf>

In [15]:
Visualization.load(f"{data_dir}/bray_curtis-ibd_sample-significance.qzv")

<visualization: Visualization uuid: b9991c38-af2b-44b4-a717-878a80f8c8a2>

In [16]:
Visualization.load(f"{data_dir}/bray_curtis-gluten_sample-significance.qzv")

<visualization: Visualization uuid: 64ae40cf-436f-4987-8bc3-20aa110c9cfc>

In [17]:
Visualization.load(f"{data_dir}/bray_curtis-continent-significance.qzv")

<visualization: Visualization uuid: 84f5b8d7-59a6-4497-aaf2-549566044cb2>

In [18]:
Visualization.load(f"{data_dir}/bray_curtis-bmi_category-significance.qzv")

<visualization: Visualization uuid: bd2d7cca-ccb0-49f6-ad39-2351a51f4c3a>

In [19]:
Visualization.load(f"{data_dir}/bray_curtis-urban_category-significance.qzv")

<visualization: Visualization uuid: 48812dc7-7f0c-44f9-b40d-eee02d566a18>

##### Adonis

In [41]:
! qiime diversity adonis \
    --i-distance-matrix $data_dir/braycurtis.qza \
    --m-metadata-file $data_dir/../Metadata/updated_fungut_metadata_noNaN.tsv \
    --p-formula "age_range + sex_sample + diet_type_sample + ibd_sample + gluten_sample + continent + bmi_category + urban_category" \
    --p-permutations 9999 \
    --o-visualization $data_dir/bray_curtis-adonis_multi.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/bray_curtis-adonis_multi.qzv


In [42]:
Visualization.load(f"{data_dir}/bray_curtis-adonis_multi.qzv")

<visualization: Visualization uuid: e751d94e-8180-48f5-8987-3c8108b6ca3a>

Notes:
- Significant different composition by continent, even after accounting for all other metadata variables
- All other metadata dont have a significant effect

In [22]:
# Check if difference between groups is due to group variance, not true group centroid separation
! qiime diversity beta-group-significance \
    --i-distance-matrix $data_dir/braycurtis.qza \
    --m-metadata-file $data_dir/../Metadata/updated_fungut_metadata.tsv \
    --m-metadata-column continent \
    --p-permutations 9999 \
    --p-method permdisp \
    --o-visualization $data_dir/braycurtis_continent_dispersion.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/braycurtis_continent_dispersion.qzv


In [23]:
Visualization.load(f"{data_dir}/braycurtis_continent_dispersion.qzv")

<visualization: Visualization uuid: edfd0916-9072-4355-92eb-0e901d1b4063>

Notes:
- Not significant -> good
- Significant PERMANOVA signal is due to differences in group centroids (actual community shifts), not because one group is just more spread out. Means each of the different groups are equally spread.

In [24]:
# Visualize possible clusterings
! qiime diversity pcoa \
    --i-distance-matrix $data_dir/braycurtis.qza \
    --o-pcoa $data_dir/braycurtis_pcoa.qza

! qiime emperor plot \
    --i-pcoa $data_dir/braycurtis_pcoa.qza \
    --m-metadata-file $data_dir/../Metadata/updated_fungut_metadata.tsv \
    --o-visualization $data_dir/braycurtis_pcoa_continent.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved PCoAResults to: Project_data/Diversity/braycurtis_pcoa.qza
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/braycurtis_pcoa_continent.qzv


In [25]:
Visualization.load(f"{data_dir}/braycurtis_pcoa_continent.qzv")

<visualization: Visualization uuid: dc839f71-9b21-45c5-934d-668b5e6426b1>

Notes:
- No clusters for continents visible, only clusters visible consists of mixed continents.

#### 3.5.2.1 Jaccard

##### Beta-group-significance

In [26]:
for col in meta_cols:
    output_name = f"{data_dir}/jaccard-{col}-significance.qzv"
    print(f"Running for column: {col}")

    ! qiime diversity beta-group-significance \
        --i-distance-matrix $data_dir/jaccard.qza \
        --m-metadata-file $data_dir/../Metadata/updated_fungut_metadata.tsv \
        --m-metadata-column {col} \
        --p-permutations 9999 \
        --p-pairwise \
        --o-visualization {output_name}
    
# Errors with country_sample (only unique values), bmi_sample (numeric type) 

Running for column: age_range
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/jaccard-age_range-significance.qzv
Running for column: sex_sample
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/jaccard-sex_sample-significance.qzv
Running for column: diet_type_sample
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_r

In [27]:
Visualization.load(f"{data_dir}/jaccard-age_range-significance.qzv")

<visualization: Visualization uuid: 15d39edb-0bfe-4d54-a800-f6c421a82bde>

In [28]:
Visualization.load(f"{data_dir}/jaccard-sex_sample-significance.qzv")

<visualization: Visualization uuid: 1034aaac-4792-4fc4-8996-7c642c74ee3b>

In [29]:
Visualization.load(f"{data_dir}/jaccard-diet_type_sample-significance.qzv")

<visualization: Visualization uuid: 2bf07a9f-6f16-4570-9f34-b60eaec9b468>

In [30]:
Visualization.load(f"{data_dir}/jaccard-ibd_sample-significance.qzv")

<visualization: Visualization uuid: b3b4d611-f720-4ca9-91b2-caa47101f540>

In [31]:
Visualization.load(f"{data_dir}/jaccard-continent-significance.qzv")

<visualization: Visualization uuid: c7e45f95-d0f8-40b7-9366-5abdb72db13e>

In [32]:
Visualization.load(f"{data_dir}/jaccard-gluten_sample-significance.qzv")

<visualization: Visualization uuid: aae03e10-af35-4530-baf6-f16584b76e58>

In [33]:
Visualization.load(f"{data_dir}/jaccard-bmi_category-significance.qzv")

<visualization: Visualization uuid: c183ae60-6565-49c6-bdd7-1f9b70f1c50a>

In [34]:
Visualization.load(f"{data_dir}/jaccard-urban_category-significance.qzv")

<visualization: Visualization uuid: 430abc5a-dd1f-49e4-b3f6-9cab2c675867>

##### Adonis

In [35]:
! qiime diversity adonis \
    --i-distance-matrix $data_dir/jaccard.qza \
    --m-metadata-file $data_dir/../Metadata/updated_fungut_metadata_noNaN.tsv \
    --p-formula "age_range + sex_sample + diet_type_sample + ibd_sample + gluten_sample + continent + bmi_category + urban_category" \
    --o-visualization $data_dir/jaccard-adonis_multi.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/jaccard-adonis_multi.qzv


In [36]:
Visualization.load(f"{data_dir}/jaccard-adonis_multi.qzv")

<visualization: Visualization uuid: 979c79e8-1ce5-429f-9505-e62506012f0f>

Notes:
- Cols with significant results from beta-group-significance:
    - Omnivore <-> Vegan
    - Vegan <-> Vegetarian
    - Europe <-> North America
    - Europe <-> Oceania
- Cols with significant results from Adoins:
    - Age_range
    - Continent
    
Means:
- Continent strongest since appears in both
- Age's effect is partially masked by other metadata
- diet_type_sample signal is confounded by other variables (continent, age, etc)

In [37]:
# Check if difference between groups is due to group variance, not true group centroid separation
! qiime diversity beta-group-significance \
    --i-distance-matrix $data_dir/jaccard.qza \
    --m-metadata-file $data_dir/../Metadata/updated_fungut_metadata.tsv \
    --m-metadata-column continent \
    --p-permutations 999 \
    --p-method permdisp \
    --o-visualization $data_dir/jaccard_continent_dispersion.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/jaccard_continent_dispersion.qzv


In [38]:
Visualization.load(f"{data_dir}/jaccard_continent_dispersion.qzv")

<visualization: Visualization uuid: 54f249cf-4786-4de2-a901-aad5db7c7518>

Notes:
- Significant -> not good
- Significant PERMANOVA signal is not due to differences in group centroids (actual community shifts), instead its because one group is just more spread out. Different spread of groups -> lead to significance in PERMANOVA.

In [39]:
# Visualize possible clusterings
! qiime diversity pcoa \
    --i-distance-matrix $data_dir/jaccard.qza \
    --o-pcoa $data_dir/jaccard_pcoa.qza

! qiime emperor plot \
    --i-pcoa $data_dir/braycurtis_pcoa.qza \
    --m-metadata-file $data_dir/../Metadata/updated_fungut_metadata.tsv \
    --o-visualization $data_dir/jaccard_pcoa_continent.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved PCoAResults to: Project_data/Diversity/jaccard_pcoa.qza
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/jaccard_pcoa_continent.qzv


In [40]:
Visualization.load(f"{data_dir}/braycurtis_pcoa_continent.qzv")

<visualization: Visualization uuid: dc839f71-9b21-45c5-934d-668b5e6426b1>

Summary:   
Overall there was no significance in alpha diversity WRONG!, thus there was no significant difference, indicating similar within-sample diversity across metadata attributes like age, continent, diet, etc.
For beta diversity bay-curtis and jaccard was examained for possible differences, it turned out jaccard's resutls are not reliable, only bay-curtis. There a significant difference between the diversity between samples originating from different continents was detected. Gut microbiome composition differs between participants from different continents.